In [ ]:
import os
import copy
import glob

import ase.io

import rmgpy.species
import rmgpy.chemkin

from hotbit import Hotbit

import autotst.species
from autotst.calculator.gaussian import Gaussian

import job_manager

## Get the species as an AutoTST Species

In this example, we load the species from a chemkin model, first as an RMG species, then convert it to an AutoTST species

In [ ]:
# load the model
chemkin_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/chem_annotated.inp"
dictionary_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/species_dictionary.txt"
transport_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/tran.dat"
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(
    chemkin_path,
    dictionary_path=dictionary_path,
    transport_path=transport_path
)
print (f"Loaded model with {len(species_list)} species and {len(reaction_list)} reactions")

In [ ]:
# TODO connect the ranked list to this code
species_index = 58  # 3 corresponds to n-heptane, the top species
spec_rmg = species_list[species_index]
spec = autotst.species.Species([spec_rmg.smiles])

print(f"loaded species {spec_rmg}")
thermo_base_dir = '/work/westgroup/harris.se/autoscience/dft/thermo'
species_base_dir = os.path.join(thermo_base_dir, f'species_{species_index:04}')
os.makedirs(species_base_dir, exist_ok=True)

# Generate Conformers

## Do a quick first pass with a cheap calculator (tight binding DFT) like Hotbit

In [ ]:
spec.generate_conformers(ase_calculator=Hotbit())
n_conformers = len(spec.conformers[spec_rmg.smiles])
print(f'{n_conformers} found with Hotbit')

## Do a more detailed calculation on the lowest energy conformers using Gaussian

In [ ]:
conformer_dir = os.path.join(species_base_dir, 'conformers')
# write Gaussian input files
for i, cf in enumerate(spec.conformers[spec_rmg.smiles]):
    gaussian = Gaussian(conformer=cf)
    calc = gaussian.get_conformer_calc()
    calc.label = f'conformer_{i:04}'

    calc.directory = conformer_dir
    calc.parameters.pop('scratch')
    calc.parameters.pop('multiplicity')
    calc.parameters['mult'] = cf.rmg_molecule.multiplicity
    
    calc.write_input(cf.ase_molecule)

print("done")

In [ ]:
# read the result back in
f = open(calc.label + ".com", "r")
print(calc.label)
print("")
for line in f.readlines():
    print(line[:-1])

In [ ]:
# Make a file to run Gaussian
slurm_run_file = os.path.join(conformer_dir, 'run.sh')
slurm_settings = {
    '--job-name': f'g16_conformers_{species_index}',
    '--error': 'error.log',
    '--output': 'output.log',
    '--nodes': 1,
    '--partition': 'west,short',
    '--exclude': 'c5003',
    '--mem': '20Gb',
    '--time': '24:00:00',
    '--cpus-per-task': 16,
    '--array': f'0-{n_conformers}%40',
}

slurm_file_writer = job_manager.SlurmJobFile(
    full_path=slurm_run_file,
)
slurm_file_writer.settings = slurm_settings

slurm_file_writer.content = [
    'export GAUSS_SCRDIR=/scratch/harris.se/guassian_scratch\n',
    'mkdir -p $GAUSS_SCRDIR\n',
    'module load gaussian/g16\n',
    'source /shared/centos7/gaussian/g16/bsd/g16.profile\n\n',

    'RUN_i=$(printf "%04.0f" $(($SLURM_ARRAY_TASK_ID)))\n',
    'fname="conformer_${RUN_i}.com"\n\n',

    'g16 $fname\n',
]
slurm_file_writer.write_file()

# submit the job
# gaussian_conformers_job = job_manager.SlurmJob()
# slurm_cmd = f"sbatch {slurm_run_file}"
# gaussian_conformers_job.submit(my_cmd)



In [ ]:
conformer_dir

In [ ]:
print(my_conformers[0].get_xyz_block())

# Arkane

In [ ]:
import autotst.calculator.statmech


In [ ]:

C5H11_1_autotst.conformers['[CH2]CCCC'][0]

In [ ]:
stat = autotst.calculator.statmech.StatMech(0)
stat.write_species_files(C5H11_1_autotst)

In [ ]:
dir(my_conformers[0])